In [1]:
!pip install psycopg2 langchain openai geopandas folium

In [2]:
import psycopg2

# データベース接続情報
host = 'postgis'
port = '5432'
user = 'postgres'
password = 'postgres'
database = 'sandbox'

# PostGISへの接続
connection = psycopg2.connect(
    host=host,
    port=port,
    user=user,
    password=password,
    database=database
)

# クエリの実行
cursor = connection.cursor()

# getTableAndColumnsName
schema = "public"
query = f"""
SELECT
  t.table_name,
  c.*
FROM
  information_schema.tables t
    JOIN information_schema.columns c
      ON t.table_name = c.table_name
WHERE
  t.table_schema = '{schema}'
    AND c.table_schema = '{schema}'
ORDER BY
  t.table_name,
  c.ordinal_position;
"""

# クエリの実行と結果の取得
cursor.execute(query)
results = cursor.fetchall()

# 結果の表示
table_info = []
for row in results:
    table_name = row[3]
    if table_name == "geography_columns":
        continue
    if table_name == "geometry_columns":
        continue
    if table_name == "spatial_ref_sys":
        continue
    column_name = row[4]
    nullable = row[7]
    data_type = row[8]
    data_type_sub = row[28]
    table_info_row = f"table_name: {table_name}, column_name: {column_name}, data_type: {data_type}/{data_type_sub}, nullable: {nullable}"
    table_info.append(table_info_row)

# コネクションのクローズ
cursor.close()
connection.close()

print("\n".join(table_info))

table_name: flood_areas_akita_2023_07_18, column_name: gridcode, data_type: bigint/int8, nullable: YES
table_name: flood_areas_akita_2023_07_18, column_name: flood_depth, data_type: text/text, nullable: YES
table_name: flood_areas_akita_2023_07_18, column_name: geometry, data_type: USER-DEFINED/geometry, nullable: YES
table_name: flood_areas_akita_2023_07_19, column_name: geometry, data_type: USER-DEFINED/geometry, nullable: YES
table_name: hospitals_akita_2023_07_18, column_name: id, data_type: bigint/int8, nullable: YES
table_name: hospitals_akita_2023_07_18, column_name: name, data_type: character varying/varchar, nullable: YES
table_name: hospitals_akita_2023_07_18, column_name: name_en, data_type: character varying/varchar, nullable: YES
table_name: hospitals_akita_2023_07_18, column_name: geom, data_type: USER-DEFINED/geometry, nullable: YES
table_name: parks, column_name: id, data_type: bigint/int8, nullable: YES
table_name: parks, column_name: name, data_type: character varying

In [3]:
import os

OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
print(len(OPENAI_API_KEY))

51


In [7]:
from langchain import OpenAI, PromptTemplate

llm = OpenAI(model_name="text-davinci-003", temperature=0)

_DEFAULT_TEMPLATE = """Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"

Only use the following tables:

{table_info}

Examples:
Question: 秋田県の浸水エリア内にある公園を教えて
SQLQuery:
```
SELECT row_to_json(featurecollection) FROM (
  SELECT
    'FeatureCollection' AS type,
    array_to_json(array_agg(feature)) AS features
  FROM (
    SELECT
      'Feature' AS type,
      ST_AsGeoJSON(geom)::json AS geometry,
      row_to_json((
        SELECT p FROM (
          SELECT p.id, p.name, p.name_en
        ) AS p
      )) AS properties
    FROM parks_akita_2023_07_18 AS p
    WHERE EXISTS (SELECT 1 FROM flood_areas_akita_2023_07_18 AS f WHERE ST_Intersects(p.geom, f.geometry))
  ) AS feature
) AS featurecollection;
```
Note: do not forget `SELECT 1` in subquery

Question: {input}"""

prompt_template = PromptTemplate(
    input_variables=["input", "table_info", "dialect"], template=_DEFAULT_TEMPLATE
)

prompt = prompt_template.format(dialect="PostGIS SQL that outputs GeoJSON", table_info="\n".join(table_info), input="秋田県の浸水エリア内にある学校を教えて", )

print("Prompt:")
print(prompt)

sql_query = llm(prompt).split("```")[1]

print("SQL Query:")
print(sql_query)

Given an input question, first create a syntactically correct PostGIS SQL that outputs GeoJSON query to run, then look at the results of the query and return the answer.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"

Only use the following tables:

table_name: flood_areas_akita_2023_07_18, column_name: gridcode, data_type: bigint/int8, nullable: YES
table_name: flood_areas_akita_2023_07_18, column_name: flood_depth, data_type: text/text, nullable: YES
table_name: flood_areas_akita_2023_07_18, column_name: geometry, data_type: USER-DEFINED/geometry, nullable: YES
table_name: flood_areas_akita_2023_07_19, column_name: geometry, data_type: USER-DEFINED/geometry, nullable: YES
table_name: hospitals_akita_2023_07_18, column_name: id, data_type: bigint/int8, nullable: YES
table_name: hospitals_akita_2023_07_18, column_name: name, data_type: character varying/varchar, nullable: YES
table_name: hospitals_akita_2023_07_18, column_name: name_en, data_type: cha

In [8]:
# PostGISへの接続
connection = psycopg2.connect(
    host=host,
    port=port,
    user=user,
    password=password,
    database=database
)

# クエリの実行
cursor = connection.cursor()

# クエリの実行と結果の取得
cursor.execute(sql_query)
results = cursor.fetchall()

# 結果の表示
print("結果:", results)

# コネクションのクローズ
cursor.close()
connection.close()

結果: [({'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[140.1466453, 39.7165407], [140.1467004, 39.7158307], [140.1467333, 39.7153782], [140.1467407, 39.7152738], [140.1467623, 39.7149932], [140.1467877, 39.7149756], [140.1476864, 39.7150159], [140.147773, 39.7150188], [140.1480088, 39.7150281], [140.1479585, 39.7157016], [140.1479501, 39.7157783], [140.1479422, 39.7158399], [140.1478786, 39.7166549], [140.1467372, 39.7165996], [140.1466453, 39.7165407]]]}, 'properties': {'id': 387752594, 'name': '秋田市立城東中学校', 'name_en': ''}}, {'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[140.1398513, 39.7161112], [140.1399626, 39.7147135], [140.1400641, 39.7146819], [140.1411615, 39.7147355], [140.1412085, 39.7147669], [140.1411051, 39.7161366], [140.1410353, 39.7161646], [140.1398513, 39.7161112]]]}, 'properties': {'id': 387752600, 'name': '秋田市立東小学校', 'name_en': ''}}]},)]


In [9]:
import folium
import geopandas as gpd

df = gpd.GeoDataFrame.from_features(results[0][0]["features"])
bounds = df.total_bounds.tolist()


m = folium.Map(
    tiles="cartodbpositron",
)

folium.GeoJson(data=results[0][0]).add_to(m)

folium.LayerControl().add_to(m)

m.fit_bounds([bounds[:2][::-1], bounds[2:][::-1]])
m